# Crear Base efecto asistencia

## Funciones

In [1]:
# Librerías de siempre
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from statsmodels.formula.api import ols
from scipy import stats

In [2]:
def formatear_base(db):
    df = db.copy()
    if 'fecha' in df.columns:
        df = df.sort_values(by=['Torneo','Round'], ascending = [True,True]).reset_index(drop=True)
    equipos = df['Local'].unique()
    equipos = pd.DataFrame(equipos, columns=['equipo'])
    equipos['i'] = equipos.index
    df = pd.merge(df, equipos, left_on='Local', right_on='equipo', how='left')
    df = df.rename(columns = {'i': 'i_local'}).drop('equipo', 1)
    df = pd.merge(df, equipos, left_on='Visita', right_on='equipo', how='left')
    df = df.rename(columns = {'i': 'i_visita'}).drop('equipo', 1)
    df = df.replace({'. Round' : ''}, regex = True)
    df['Round'] = df['Round'].astype(int)
    df['goles L'] = df['goles L'].astype(int)
    df['goles V'] = df['goles V'].astype(int)
    return df

In [3]:
def tabla_final_torneo(db):
    df = db.copy()
    if 'i_local' not in df.columns:
        df = formatear_base(df)
    tabla = df[['Local','i_local']].drop_duplicates()
    tabla = tabla.set_index(['i_local'])
    tabla.columns = ['equipo']
    conditions = [
            (df['goles L'] > df['goles V']),
            (df['goles L'] < df['goles V'])]
    choices = ['local', 'visita']
    df = df.join(pd.get_dummies(np.select(conditions, choices, default = 'empate')))
    ghome = df.groupby('i_local')
    gaway = df.groupby('i_visita')
    df_home = pd.DataFrame({'wins_h': ghome['local'].sum(),
                            'draws_h': ghome['empate'].sum(),
                            'losses_h': ghome['visita'].sum(),
                            'gf_h': ghome['goles L'].sum(),
                            'ga_h': ghome['goles V'].sum(),
                            'gd_h': ghome['goles L'].sum() - ghome['goles V'].sum()})
    df_away = pd.DataFrame({'wins_a': gaway['visita'].sum(),
                            'draws_a': gaway['empate'].sum(),
                            'losses_a': gaway['local'].sum(),
                            'gf_a': gaway['goles V'].sum(),
                            'ga_a': gaway['goles L'].sum(),
                            'gd_a': gaway['goles V'].sum() - gaway['goles L'].sum()})
    tabla = tabla.join(df_home, how='left').join(df_away,how = 'left').fillna(0)
    tabla['wins'] = tabla.wins_h + tabla.wins_a
    tabla['draws'] = tabla.draws_h + tabla.draws_a
    tabla['losses'] = tabla.losses_h + tabla.losses_a
    tabla['gf'] = tabla.gf_h + tabla.gf_a
    tabla['ga'] = tabla.ga_h + tabla.ga_a
    tabla['gd'] = tabla.gd_h + tabla.gd_a
    tabla['points'] = (tabla['wins']*3 + tabla['draws']).astype(int)
    tabla = tabla.sort_values(by=['points','gd'], ascending = False).reset_index(drop=True)
    tabla['position'] = (tabla.index + 1).astype(int)
    return tabla[['equipo','wins','draws','losses','gf','ga','gd','points','position']]

In [4]:
def tabla_final_local(db):
    df = db.copy()
    if 'i_local' not in df.columns:
        df = formatear_base(df)
    tabla = df[['Local','i_local']].drop_duplicates()
    tabla = tabla.set_index(['i_local'])
    tabla.columns = ['equipo']
    conditions = [
            (df['goles L'] > df['goles V']),
            (df['goles L'] < df['goles V'])]
    choices = ['local', 'visita']
    df = df.join(pd.get_dummies(np.select(conditions, choices, default = 'empate')))
    ghome = df.groupby('i_local')
    df_home = pd.DataFrame({'wins': ghome['local'].sum(),
                            'draws': ghome['empate'].sum(),
                            'losses': ghome['visita'].sum(),
                            'gf': ghome['goles L'].sum(),
                            'ga': ghome['goles V'].sum(),
                            'gd': ghome['goles L'].sum() - ghome['goles V'].sum()})
    tabla = tabla.join(df_home, how='left').fillna(0)
    tabla['points'] = (tabla['wins']*3 + tabla['draws']).astype(int)
    #Normalizar puntos según cantidad de partidos jugados de local
    tabla['norm_points'] = tabla['points']/(tabla['wins'] + tabla['draws'] + tabla['losses']) 
    tabla = tabla.sort_values(by=['norm_points','points','gd'], ascending = False).reset_index(drop=True)
    tabla['position'] = (tabla.index + 1).astype(int)
    return tabla[['equipo','wins','draws','losses','gf','ga','gd','points','position']]

In [5]:
def tabla_final_visita(db):
    df = db.copy()
    if 'i_visita' not in df.columns:
        df = formatear_base(df)
    tabla = df[['Visita','i_visita']].drop_duplicates()
    tabla = tabla.set_index(['i_visita'])
    tabla.columns = ['equipo']
    conditions = [
            (df['goles L'] > df['goles V']),
            (df['goles L'] < df['goles V'])]
    choices = ['local', 'visita']
    df = df.join(pd.get_dummies(np.select(conditions, choices, default = 'empate')))
    gaway = df.groupby('i_visita')
    df_away = pd.DataFrame({'wins': gaway['visita'].sum(),
                            'draws': gaway['empate'].sum(),
                            'losses': gaway['local'].sum(),
                            'gf': gaway['goles V'].sum(),
                            'ga': gaway['goles L'].sum(),
                            'gd': gaway['goles V'].sum() - gaway['goles L'].sum()})
    tabla = tabla.join(df_away, how='left').fillna(0)
    tabla['points'] = (tabla['wins']*3 + tabla['draws']).astype(int)
    #Normalizar puntos según cantidad de partidos jugados de local
    tabla['norm_points'] = tabla['points']/(tabla['wins'] + tabla['draws'] + tabla['losses']) 
    tabla = tabla.sort_values(by=['norm_points','points','gd'], ascending = False).reset_index(drop=True)
    tabla['position'] = (tabla.index + 1).astype(int)
    return tabla[['equipo','wins','draws','losses','gf','ga','gd','points','position']]

In [26]:
def crear_binarias_indicadores(df,torneo_actual,torneo_anterior, ptos_corte = [3,4,5], rondas_corte = [4,5,6,7]):
    """
    Crea las columnas de indicadores
    """
    columnas_base = df.columns.tolist()
    t_ant = tabla_final_torneo(df[df['Torneo'] == torneo_anterior])
    t_ant_local = tabla_final_local(df[df['Torneo'] == torneo_anterior])
    t_ant_visita = tabla_final_visita(df[df['Torneo'] == torneo_anterior])
    df_actual = formatear_base(df[df['Torneo'] == torneo_actual]).sort_values(by=['Round'], ascending = True).reset_index(drop=True)

    eq_act = df_actual['Local'].drop_duplicates().tolist()
    eq_ant_l =  t_ant_local['equipo'].drop_duplicates().tolist()
    eq_ant_v =  t_ant_visita['equipo'].drop_duplicates().tolist()
    eq_ant = t_ant['equipo'].drop_duplicates().tolist() #esta ya está ordenada por posición
    eq_correc = [i for i in eq_ant if i in eq_act] + [j for j in eq_act if j not in eq_ant]
    eq_correc_l = [i for i in eq_ant_l if i in eq_act] + [j for j in eq_act if j not in eq_ant_l]
    eq_correc_v = [i for i in eq_ant_v if i in eq_act] + [j for j in eq_act if j not in eq_ant_v]  
    t_ant_local = t_ant_local[t_ant_local['equipo'].isin(eq_act)].reset_index(drop=True)
    t_ant_local['position'] = t_ant_local.index + 1
    t_ant_visita = t_ant_visita[t_ant_visita['equipo'].isin(eq_act)].reset_index(drop=True)
    t_ant_visita['position'] = t_ant_visita.index + 1
    posicion_nuevos = t_ant_local['position'].max() + 1
    position = [min(i+1,posicion_nuevos) for i in range(len(eq_correc))]
    t_ant = pd.DataFrame({'equipo': eq_correc,
                      'position': position})
    t_ant_local = pd.DataFrame({'equipo': eq_correc_l,
                          'position': position})
    t_ant_visita = pd.DataFrame({'equipo': eq_correc_v,
                          'position': position})
    t_ant_ultimo = t_ant['position'].astype(int).max()

    """
    SIGNIFICADO VARIABLES


    f_gral_visita: 1 si el equipo visitante es fácil según la tabla general
    del torneo anterior, 0 si no
    d_gral_visita: 1 si el equipo visitante es difícil según la tabla general
    del torneo anterior, 0 si no
    f_gral_local: 1 si el equipo local es fácil según la tabla general
    del torneo anterior, 0 si no
    d_gral_local: 1 si el equipo local es difícil según la tabla general
    del torneo anterior, 0 si no

    f_visita: 1 si el equipo visitante es fácil jugando de visita según
    la tabla de posiciones de visita, 0 si no
    d_visita: 1 si el equipo visitante es difícil jugando de visita según
    la tabla de posiciones de visita, 0 si no
    f_local: 1 si equipo local es fácil jugando de local según
    la tabla de posiciones de local, 0 si no
    d_local: 1 si el equipo local es difícil jugando de local según
    la tabla de posiciones de local, 0 si no

    f_..._corr es lo mismo, salvo que se hacre la corrección de pertenecer
    al mismo grupo

    posicion_gral_visita: posición del equipo visitante según la tabla general del
    torneo anterior
    posicion_gral_local: posición del equipo local según la tabla general del
    torneo anterior
    posicion_visita: posición del equipo visitante según la tabla de visita del
    torneo anterior
    posicion_local: posición del equipo local según la tabla de local del
    torneo anterior



    """


    for r in rondas_corte:
        df_actual['p%s' % r] = np.where(df_actual['Round'] <= r, 1, 0)
        if r == 3:
            df_actual['posicion_gral_visita'] = df_actual['Visita'].map(dict_eq_pos_gral)
            df_actual['posicion_gral_local'] = df_actual['Local'].map(dict_eq_pos_gral)
            df_actual['posicion_visita'] = df_actual['Visita'].map(dict_eq_pos_visita)
            df_actual['posicion_local'] = df_actual['Local'].map(dict_eq_pos_local)

    for c in ptos_corte:
        equipos_5p = t_ant[t_ant['position'] <= c]['equipo'].values.tolist()
        local_5p = t_ant_local[t_ant_local['position'] <= c]['equipo'].values.tolist()
        visita_5p = t_ant_visita[t_ant_visita['position'] <= c]['equipo'].values.tolist()
        equipos_5u = t_ant[t_ant['position'] >= t_ant_ultimo - c + 1]['equipo'].values.tolist()
        local_5u = t_ant_local[t_ant_local['position'] >= t_ant_ultimo - c + 1]['equipo'].values.tolist() 
        visita_5u = t_ant_visita[t_ant_visita['position'] >= t_ant_ultimo - c + 1]['equipo'].values.tolist()

        dict_eq_pos_gral = dict(zip(t_ant['equipo'].values, t_ant['position'].values))
        dict_eq_pos_local = dict(zip(t_ant_local['equipo'].values, t_ant_local['position'].values))
        dict_eq_pos_visita = dict(zip(t_ant_visita['equipo'].values, t_ant_visita['position'].values))
        
        df_actual['f_gral_visita_c%s' % c] = np.where(df_actual['Visita'].isin(equipos_5u), 1, 0)
        df_actual['d_gral_visita_c%s' % c] = np.where(df_actual['Visita'].isin(equipos_5p), 1, 0)
        df_actual['f_gral_local_c%s' % c] = np.where(df_actual['Local'].isin(equipos_5u), 1, 0)
        df_actual['d_gral_local_c%s' % c] = np.where(df_actual['Local'].isin(equipos_5p), 1, 0)    

        df_actual['f_visita_c%s' % c] = np.where(df_actual['Visita'].isin(visita_5u), 1, 0)
        df_actual['d_visita_c%s' % c] = np.where(df_actual['Visita'].isin(visita_5p), 1, 0)
        df_actual['f_local_c%s' % c] = np.where(df_actual['Local'].isin(local_5u), 1, 0)
        df_actual['d_local_c%s' % c] = np.where(df_actual['Local'].isin(visita_5p), 1, 0)
    col_basicas = ['Local','Visita','Torneo','Round','Stadium','Attendance','Capacity']
    col_nuevas = [i for i in df_actual.columns if i not in columnas_base and 'i_' not in i]
    return df_actual[col_basicas + col_nuevas]

In [7]:
def base_efecto_secuencial(df, filtro_ronda = False, ronda_corte = 5, pto_corte = 5, replace_releg = False):
    Torneos = df['Torneo'].drop_duplicates().values.tolist()
    dbs = []
    max_ronda = []
    for i in range(1,len(Torneos)):
        df_actual = bases_pre_indicadores(df,Torneos[i],Torneos[i-1], pto_corte = pto_corte, replace_releg = replace_releg)
        dbs.append(variables_efecto_secuencial(df_actual, ronda_corte = ronda_corte))
        max_ronda.append(formatear_base(df[df['Torneo'] == Torneos[i]])['Round'].max())
    db = pd.concat(dbs, ignore_index=True)
    if filtro_ronda:
        return db[db['position'] <= min(max_ronda)].reset_index(drop=True)
    else:
        return db

In [32]:
datadir

'..\\Datos\\Ligas ex'

In [8]:
def grabar_bd(datadir,nombre_base, ronda_corte = 5, pto_corte = 5, replace_releg = False):
    dfs = []
    sheets = []
    for subdir, dirs, files in os.walk(datadir):
        for file in files:
            filepath = subdir  + file
            if filepath.endswith(".xlsx"):
                df = pd.read_excel(filepath)
                dfs.append(base_efecto_secuencial(df, ronda_corte = ronda_corte,
                                                  pto_corte = pto_corte,
                                                  replace_releg = replace_releg).round(3))
                sheets.append(file[:-5])
                
    if ".xlsx" not in nombre_base:
        nombre_base = nombre_base + ".xlsx"
    writer = pd.ExcelWriter(nombre_base,engine='xlsxwriter')   
    for dataframe, sheet in zip(dfs, sheets):
        dataframe.to_excel(writer, sheet_name=sheet, index = False)   
    writer.save()
    return dfs, sheets

## Preliminares

Ordenar las bases de ligas por fecha y eliminar el último torneo.

In [10]:
def ordenar_bases_ligas(datadir):
    for subdir, dirs, files in os.walk(datadir):
        for file in files:
            if file.endswith('.xlsx'):
                print('Leyendo: %s               ' % file, end = '\t\r')
                df = pd.read_excel(os.path.join(datadir,file))
                df = df.sort_values(by = ['Date'], ascending = True)
                Torneos = df['Torneo'].drop_duplicates().tolist()
                df = df[df['Torneo'].isin(Torneos[:-1])]
                df.to_excel(os.path.join(datadir,file), index = False)

## Ejecución

In [44]:
def efecto_asistencia(datadir,outputdir):
    for subdir, dirs, files in os.walk(datadir):
        for file in files:
            if file.endswith('.xlsx'):
                print('Construyendo: %s          ' % file)
                df_test = pd.read_excel(os.path.join(subdir, file))
                Torneos_test = df_test['Torneo'].drop_duplicates().tolist()
                dfs_test = []
                for i in range(1,len(Torneos_test)):
                    torneo_actual = Torneos_test[i]
                    torneo_anterior = Torneos_test[i-1]
                    dfs_test.append(crear_binarias_indicadores(df_test,torneo_actual,torneo_anterior))
                    df_test2 = pd.concat(dfs_test, ignore_index = True)
                    cols = ['Attendance','Capacity']
                    df_test2[cols] = df_test2[cols].replace({0:np.nan})
                    df_test2 = df_test2.dropna(subset=cols)
                    df_test2['Occupation'] = np.minimum(df_test2['Attendance']/df_test2['Capacity'],1)*100
                    col_basicas = ['Attendance','Capacity','Occupation','Local','Visita','Torneo','Round','Stadium']
                    df_test2 = df_test2[col_basicas + [i for i in df_test2.columns if i not in col_basicas]]
                    df_test2.to_excel(os.path.join(outputdir, file), index = False)

In [46]:
df_test = pd.read_excel(os.path.join(outputdir,'eng-premier-league.xlsx'))
df_test.head()

,Attendance,Capacity,Occupation,Local,Visita,Torneo,Round,Stadium,p4,p5,...,f_local_c4,d_local_c4,f_gral_visita_c5,d_gral_visita_c5,f_gral_local_c5,d_gral_local_c5,f_visita_c5,d_visita_c5,f_local_c5,d_local_c5
0,33033,40242,82.085880,Leeds United,Newcastle United,Premier League 1956/1957,35,Elland Road,0,0,...,1,0,0,0,1,0,0,0,1,0
1,37907,54000,70.198148,Sunderland AFC,Preston North End,Premier League 1956/1957,35,Roker Park,0,0,...,0,0,1,0,0,0,0,1,0,0
2,17610,16000,100.000000,Blackpool FC,Birmingham City,Premier League 1956/1957,35,Bloomfield Road,0,0,...,0,1,0,0,0,1,0,0,0,1
3,36110,36284,99.520450,Tottenham Hotspur,Portsmouth FC,Premier League 1956/1957,35,White Hart Lane,0,0,...,1,0,0,0,1,0,0,0,1,0
4,18424,60000,30.706667,Bolton Wanderers,Sheffield Wednesday,Premier League 1956/1957,35,Burnden Park,0,0,...,0,0,1,0,0,0,1,0,0,0


In [52]:
formula = 'Occupation ~ Stadium + f_local_c5 + d_local_c5 + f_visita_c5 + d_visita_c5'
mod = ols(formula = formula, data = df_test).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:             Occupation   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     192.0
Date:                Sun, 10 Mar 2019   Prob (F-statistic):               0.00
Time:                        19:50:25   Log-Likelihood:            -1.1001e+05
No. Observations:               26053   AIC:                         2.202e+05
Df Residuals:                   25972   BIC:                         2.209e+05
Df Model:                          80                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------